In [1]:
import os
import pandas as pd
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:13 http://ppa.launchpad.net/

In [2]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import HashingTF
import re

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-18 23:54:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-18 23:54:36 (6.41 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("capstone-project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
dataset_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fake_job_postings.csv")
dataset_df.head(6)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,US; NY; New York,Marketing,NaN,We're Food52 and we've created a groundbreakin...,Food52 a fast-growing James Beard Award-winnin...,Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,NZ; ; Auckland,Success,NaN,90 Seconds the worlds Cloud Video Production S...,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),US; IA; Wever,NaN,NaN,Valor Services provides Workforce Solutions th...,Our client located in Houston is actively seek...,Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,US; DC; Washington,Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,EDUCATION: Bachelor’s or Master’s in GIS busin...,Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,US; FL; Fort Worth,NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
5,6,Accounting Clerk,US; MD;,NaN,NaN,NaN,Job OverviewApex is an environmental consultin...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0


In [6]:
dataset_df = dataset_df.drop(columns=['job_id','salary_range'])

In [7]:
dataset_df.isnull().sum()

title                      0
location                 346
department             11547
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [8]:
location_split = dataset_df['location'].str.split(';', n=2, expand=True)
dataset_df = dataset_df.join(location_split)
dataset_df.head(1)

,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,0,1,2
0,Marketing Intern,US; NY; New York,Marketing,We're Food52 and we've created a groundbreakin...,Food52 a fast-growing James Beard Award-winnin...,Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,US,NY,New York


In [9]:
dataset_df.rename(columns={0: "Country", 1: "City", 2: "State"},inplace=True)
dataset_df = dataset_df.drop(columns=['location'])
dataset_df.head(1)

,title,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,City,State
0,Marketing Intern,Marketing,We're Food52 and we've created a groundbreakin...,Food52 a fast-growing James Beard Award-winnin...,Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,US,NY,New York


In [10]:
dataset_df.dtypes

title                  object
department             object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
Country                object
City                   object
State                  object
dtype: object

In [11]:
clean_cols = ["company_profile", "description", "requirements", "benefits"]
str_cols = dataset_df.select_dtypes(['object'])

for col in clean_cols:
  dataset_df[col] = dataset_df[col].replace(r'[^a-zA-Z0-9\s]', '',regex=True)
  dataset_df[col] = dataset_df[col].replace(r'\s{2,}', '',regex=True)

#dataset_df[str_cols.columns] = str_cols.apply(lambda x: x.str.replace(r'[^a-zA-Z0-9\s]', '',regex=True))
#dataset_df[str_cols.columns] = str_cols.apply(lambda x: x.str.replace(r'\-', ' ',regex=True))
#
dataset_df.head(6)

,title,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,City,State
0,Marketing Intern,Marketing,Were Food52 and weve created a groundbreaking ...,Food52 a fastgrowing James Beard Awardwinning ...,Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,US,NY,New York
1,Customer Service - Cloud Video Production,Success,90 Seconds the worlds Cloud Video Production S...,OrganisedFocusedVibrantAwesomeDo you have a pa...,What we expect from youYour key responsibility...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,NZ,,Auckland
2,Commissioning Machinery Assistant (CMA),NaN,Valor Services provides Workforce Solutions th...,Our client located in Houston is actively seek...,Implement precommissioning and commissioning p...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0,US,IA,Wever
3,Account Executive - Washington DC,Sales,Our passion for improving quality of life thro...,THE COMPANY ESRIEnvironmental Systems Research...,EDUCATION Bachelors or Masters in GIS business...,Our culture is anything but corporatewe have a...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,Washington
4,Bill Review Manager,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE Itemization Review ManagerLOCATION F...,QUALIFICATIONSRN license in the State of Texas...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,Fort Worth
5,Accounting Clerk,NaN,NaN,Job OverviewApex is an environmental consultin...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0,US,MD,


In [12]:
dataset_df = dataset_df.fillna("none")
dataset_df.head()

,title,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country,City,State
0,Marketing Intern,Marketing,Were Food52 and weve created a groundbreaking ...,Food52 a fastgrowing James Beard Awardwinning ...,Experience with content management systems a m...,none,0,1,0,Other,Internship,none,none,Marketing,0,US,NY,New York
1,Customer Service - Cloud Video Production,Success,90 Seconds the worlds Cloud Video Production S...,OrganisedFocusedVibrantAwesomeDo you have a pa...,What we expect from youYour key responsibility...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,none,Marketing and Advertising,Customer Service,0,NZ,,Auckland
2,Commissioning Machinery Assistant (CMA),none,Valor Services provides Workforce Solutions th...,Our client located in Houston is actively seek...,Implement precommissioning and commissioning p...,none,0,1,0,none,none,none,none,none,0,US,IA,Wever
3,Account Executive - Washington DC,Sales,Our passion for improving quality of life thro...,THE COMPANY ESRIEnvironmental Systems Research...,EDUCATION Bachelors or Masters in GIS business...,Our culture is anything but corporatewe have a...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,Washington
4,Bill Review Manager,none,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE Itemization Review ManagerLOCATION F...,QUALIFICATIONSRN license in the State of Texas...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,Fort Worth


In [13]:
dataset_df.isnull().sum()

title                  0
department             0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
Country                0
City                   0
State                  0
dtype: int64

In [14]:
df = spark.createDataFrame(dataset_df)

In [15]:
df.show()

+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+
|               title|department|     company_profile|         description|        requirements|            benefits|telecommuting|has_company_logo|has_questions|employment_type|required_experience|  required_education|            industry|            function|fraudulent|Country|City|            State|
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+
|    Marketing Intern| Marketing|Were Food52 and w...|Food52 a fastgrow...|Experience wi

In [16]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.sql.functions import length
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline

clean_columns = ["company_profile", "description", "requirements", "benefits"]
data_df = df
for col in clean_columns:
  # Create a length column to be used as a future feature
  data_df = data_df.withColumn(f'{col}_length', length(df[col]))

data_df.show()


+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+----------------------+------------------+-------------------+---------------+
|               title|department|     company_profile|         description|        requirements|            benefits|telecommuting|has_company_logo|has_questions|employment_type|required_experience|  required_education|            industry|            function|fraudulent|Country|City|            State|company_profile_length|description_length|requirements_length|benefits_length|
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+-----------------

In [17]:
 cleaned_df = data_df
 for col in clean_columns: 
  # Create all the features to the data set
  tokenizer = Tokenizer(inputCol=col, outputCol=f'{col}_token_text')
  stopremove = StopWordsRemover(inputCol=f'{col}_token_text',outputCol=f'{col}_stop_tokens')
  hashingTF = HashingTF(inputCol=f'{col}_stop_tokens', outputCol=f'{col}_hash_token')
  idf = IDF(inputCol=f'{col}_hash_token', outputCol=f'{col}_idf_token')

  # Create feature vectors
  clean_up = VectorAssembler(inputCols=[f'{col}_idf_token', f'{col}_length'], outputCol=f'{col}_features')
  data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

  cleaner = data_prep_pipeline.fit(cleaned_df)
  cleaned_df = cleaner.transform(cleaned_df)

cleaned_df.show(truncate=True)

+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+----------------------+------------------+-------------------+---------------+--------------------------+---------------------------+--------------------------+-------------------------+------------------------+----------------------+-----------------------+----------------------+---------------------+--------------------+-----------------------+------------------------+-----------------------+----------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|department|     company_profile|         description|        requirements|            benefits|telecommuting|has_company_logo|h

In [18]:
ml_df = cleaned_df.select('title','department','telecommuting','has_company_logo','has_questions','employment_type','required_experience','industry','function','Country','City','State','company_profile_features','description_features','requirements_features','benefits_features','fraudulent')
ml_df.show()

+--------------------+----------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+-------+----+-----------------+------------------------+--------------------+---------------------+--------------------+----------+
|               title|department|telecommuting|has_company_logo|has_questions|employment_type|required_experience|            industry|            function|Country|City|            State|company_profile_features|description_features|requirements_features|   benefits_features|fraudulent|
+--------------------+----------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+-------+----+-----------------+------------------------+--------------------+---------------------+--------------------+----------+
|    Marketing Intern| Marketing|            0|               1|            0|          Other|         Internship|                none| 

In [ ]:
data = ml_df.toPandas()
# store the feature matrix (X) and response vector (y)
X = data.drop('fraudulent')
y = data.fraudulent
 
# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)
 
# training the model on training set
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
 
# making predictions on the testing set
y_pred = gnb.predict(X_test)
 
# comparing actual response values (y_test) with predicted response values (y_pred)
from sklearn import metrics
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

In [ ]:
# Break data down into a training set and a testing set
training, testing = ml_df.randomSplit([0.7, 0.3], 21)

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
nb.featuresCol = training['fraudulent']
predictor = nb.fit(training)

IllegalArgumentException: ignored

In [ ]:
test_results = predictor.transform(testing)
test_results.show(5)

In [ ]:
# OLD CODE

In [ ]:
tockenize_cols = ["company_profile", "description", "requirements", "benefits"]
# Tokenize sentences
tockenized_df = df

for col in tockenize_cols:
  tokenizer = Tokenizer(inputCol=col, outputCol=f'{col}_tokens')
  tockenized_df = tokenizer.transform(tockenized_df)
  tockenized_df = tockenized_df.drop(col)
  tockenized_df = tockenized_df.withColumnRenamed(f'{col}_tokens',col)

tockenized_df.show()

+--------------------+----------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+--------------------+--------------------+--------------------+--------------------+
|               title|department|telecommuting|has_company_logo|has_questions|employment_type|required_experience|  required_education|            industry|            function|fraudulent|Country|City|            State|     company_profile|         description|        requirements|            benefits|
+--------------------+----------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+--------------------+--------------------+--------------------+--------------------+
|    Marketing Intern| Marketing|            0|               1|            0|          

In [ ]:
for col in tockenize_cols:
  remover = StopWordsRemover(inputCol=col, outputCol=f'{col}_stopw')
  tockenized_df = remover.transform(tockenized_df)
  tockenized_df = tockenized_df.drop(col)
  tockenized_df = tockenized_df.withColumnRenamed(f'{col}_stopw',col)

tockenized_df.show()

+--------------------+----------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+--------------------+--------------------+--------------------+--------------------+
|               title|department|telecommuting|has_company_logo|has_questions|employment_type|required_experience|  required_education|            industry|            function|fraudulent|Country|City|            State|     company_profile|         description|        requirements|            benefits|
+--------------------+----------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+----+-----------------+--------------------+--------------------+--------------------+--------------------+
|    Marketing Intern| Marketing|            0|               1|            0|          

In [ ]:
hashing = HashingTF(inputCol='description', outputCol='hashedValues', numFeatures=pow(2,18))

hashed_df = hashing.transform(tockenized_df)
hashed_df.show(truncate=False)

+------------------------------------------------------------+----------+-------------+----------------+-------------+---------------+-------------------+-------------------------+-----------------------------------+----------------------+----------+-------+----+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
idf = IDF(inputCol='hashedValues', outputCol='features')
idfModel = idf.fit(hashed_df)
rescaleData = idfModel.transform(hashed_df)

rescaleData.select('description','features').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
result = tockenized_df.withColumn('word', f.explode(f.col('description'))) \
  .groupBy('word') \
  .count().sort('count', ascending=False) \

result.show(500)

+----------------+-----+
|            word|count|
+----------------+-----+
|            team|16469|
|            work|13532|
|        business|10038|
|             new| 9572|
|        customer| 9050|
|           sales| 8812|
|      experience| 8624|
|         looking| 8609|
|         company| 8049|
|     development| 7557|
|         product| 6979|
|      management| 6802|
|         service| 6377|
|       marketing| 6325|
|             amp| 6145|
|         support| 5967|
|          design| 5576|
|         clients| 5513|
|            data| 5385|
|       customers| 5308|
|         working| 5284|
|        services| 5256|
|        position| 5038|
|          client| 5031|
|             job| 4804|
|            role| 4790|
|       including| 4548|
|         project| 4489|
|          skills| 4277|
|        software| 4271|
|            help| 4214|
|     opportunity| 4096|
|         quality| 4079|
|        products| 4040|
|              us| 3888|
|         provide| 3862|
|            well| 3841|


In [ ]:
from nltk import tokenize
from operator import itemgetter
import math